In [21]:
import sympy as sym
from sympy.abc import x, y, z
from sympy import Poly
from sympy import LT, LM, LC
from sympy import lcm
from sympy import reduced

#WE CAN CHECK myGroebner WITH sym.groebner(f, x, y, z, order = "lex"), USE SUITABLE GENS AND ORDER

In [22]:
def myDiv(lst, f, t, gen, dom):
    a = [0] * len(lst)
    r = 0
    p = f
    while p != 0:
        i = 0
        divisionoccured = False
        while i<len(lst) and divisionoccured == False:
            if sym.div(Poly(LT(p, order = t), gen, domain = dom), Poly(LT(lst[i], order = t), gen, domain = dom))[1] == 0 :
                a[i] = a[i] + sym.div(Poly(LT(p, order = t), gen, domain = dom), Poly(LT(lst[i], order = t), gen, domain = dom))[0]
                p = p - sym.div(Poly(LT(p, order = t), gen, domain = dom), Poly(LT(lst[i], order = t), gen, domain = dom))[0] * lst[i]
                divisionoccured = True
            else:
                i = i + 1
        if divisionoccured == False:
            r = r + LT(p, order = t)
            p = p - LT(p, order = t)
    return a, r

In [23]:
def calS(f, g, t):
    x_gamma = lcm(LM(f, order = t), LM(g, order = t))
    S = x_gamma/LT(f, order = t) * f - x_gamma/LT(g, order = t) * g
    return S

In [24]:
def myGroebner(f, t, gen, dom):
    g = f.copy()
    rolling = True
    while rolling == True:
        g_control = g.copy()
        for i in range (len(g_control)-1):
            for j in range (i+1, len(g_control)):
                S = calS(g[i], g[j], t = t)
                rem = myDiv(g_control, S, t = t, gen = gen, dom = dom)[1]
                if rem !=0:
                    g.append(Poly(rem, gen, domain = dom))
        if g == g_control:
            rolling = False

    for i in range (len(g)):
        print(f"g{i} = {sym.latex(g[i].as_expr())}")
        
    return g

In [25]:
def myGroebnerCheck(g, t, gen, dom):
    dict_of_S={}
    for i in range(len(g) - 1):
        for j in range(i+1, len(g)):
            S = calS(g[i], g[j], t = t)      
            dict_of_S[i,j] = S

    for item in dict_of_S:
        print(f"S(g{item[0]}, g{item[1]}) = {sym.latex(dict_of_S[item].as_expr())}")
        print(f"Reminder of division with g = {myDiv(g, dict_of_S[item], t = t, gen = gen, dom = dom)[1]}")
        
    return

In [26]:
def myRedGroeb(f, t, gen, dom):
    #HAVING GROEBNER BASIS
    g = myGroebner(f, t = t, gen = gen, dom = dom)

    #HAVING MONIC GROEBNER BASIS
    for i in range(len(g)):
        g[i] = sym.monic(g[i])

    #MAKING LIST OF LT(g):
    lstLT = []
    for i in range (len(g)):
        lstLT.append(Poly(LT(g[i], order = t), gen, domain = dom))

    #CHANGING THE NON REDUCED BASIS INTO ZERO
    h = g.copy()
    for i in range (len(g)):
        while h[i] != 0 :
            denum = lstLT[0:i] + lstLT[i+1:]
            rem = myDiv(denum, Poly(LT(h[i], order = t), gen, domain = dom), t = t, gen = gen, dom = dom)[1]
            if rem == 0 :
                g[i] = 0
                break
            else:
                h[i] = h[i] - LT(h[i], order = t)

    #REMOVING THE ZERO BASIS
    g = list( filter( lambda a: a != 0, g ) )
    print(f"Reduced Groebner = {g}")
    return g

In [27]:
#01

f0 = Poly(-x**3 + y, x, y, z, domain = 'QQ')
f1 = Poly(x**2*y - z, x, y, z, domain = 'QQ')

f = [f0, f1]

print("---reduced Groebner---")
G = myRedGroeb(f, t = "lex", gen =(x,y,z), dom='QQ')
print("---with sympy---")
f = [f0, f1]
realG = sym.groebner(f, x, y, z, order = "lex")
print(realG)

---reduced Groebner---
g0 = - x^{3} + y
g1 = x^{2} y - z
g2 = x z - y^{2}
g3 = x y^{3} - z^{2}
g4 = - y^{5} + z^{3}
Reduced Groebner = [Poly(x**3 - y, x, y, z, domain='QQ'), Poly(x**2*y - z, x, y, z, domain='QQ'), Poly(x*z - y**2, x, y, z, domain='QQ'), Poly(x*y**3 - z**2, x, y, z, domain='QQ'), Poly(y**5 - z**3, x, y, z, domain='QQ')]
---with sympy---
GroebnerBasis([x**3 - y, x**2*y - z, x*y**3 - z**2, x*z - y**2, y**5 - z**3], x, y, z, domain='QQ', order='lex')


In [44]:
# 02

f0 = Poly(x*z - y, x, y, z, domain = 'QQ')
f1 = Poly(x*y + 2*z**2, x, y, z, domain = 'QQ')
f2 = Poly(y - z, x, y, z, domain = 'QQ')

f = [f0, f1, f2]
print(f)

print("\n--Groebner by sympy--")
realG = sym.groebner(f, x, y, z, order = "lex", domain="QQ")
print(realG)

print("\n--membership of realG in f--")
print(reduced(realG[0], f, domain="QQ"))
print(reduced(realG[1], f, domain="QQ"))
print(reduced(realG[2], f, domain="QQ")) #NOT GIVING ZERO REMINDER !!!

print("\n--membership of f in realG--")
print(reduced(f[0], realG))
print(reduced(f[1], realG))
print(reduced(f[2], realG))

[Poly(x*z - y, x, y, z, domain='QQ'), Poly(x*y + 2*z**2, x, y, z, domain='QQ'), Poly(y - z, x, y, z, domain='QQ')]

--Groebner by sympy--
GroebnerBasis([x*z - z, y - z, z**2 + z/2], x, y, z, domain='QQ', order='lex')

--membership of realG in f--
([Poly(1, x, y, z, domain='QQ'), Poly(0, x, y, z, domain='QQ'), Poly(1, x, y, z, domain='QQ')], Poly(0, x, y, z, domain='QQ'))
([Poly(0, x, y, z, domain='QQ'), Poly(0, x, y, z, domain='QQ'), Poly(1, x, y, z, domain='QQ')], Poly(0, x, y, z, domain='QQ'))
([Poly(0, x, y, z, domain='QQ'), Poly(0, x, y, z, domain='QQ'), Poly(0, x, y, z, domain='QQ')], Poly(z**2 + 1/2*z, x, y, z, domain='QQ'))

--membership of f in realG--
([Poly(1, x, y, z, domain='QQ'), Poly(-1, x, y, z, domain='QQ'), Poly(0, x, y, z, domain='QQ')], Poly(0, x, y, z, domain='QQ'))
([Poly(1, x, y, z, domain='QQ'), Poly(x, x, y, z, domain='QQ'), Poly(2, x, y, z, domain='QQ')], Poly(0, x, y, z, domain='QQ'))
([Poly(0, x, y, z, domain='QQ'), Poly(1, x, y, z, domain='QQ'), Poly(0, x, y

In [36]:
f0 = Poly(x*z - y, x, y, z, domain = 'QQ')
f1 = Poly(x*y + 2*z**2, x, y, z, domain = 'QQ')
f2 = Poly(y - z, x, y, z, domain = 'QQ')

f = [f0, f1, f2]
temp = calS(f0,f1,t="lex")
myDiv(f, temp, t="lex", gen=(x,y,z), dom="QQ")

([0, 0, Poly(-y - z, x, y, z, domain='QQ')], -2*z**3 - z**2)

In [37]:
f0 = Poly(x*z - y, x, y, z, domain = 'QQ')
f1 = Poly(x*y + 2*z**2, x, y, z, domain = 'QQ')
f2 = Poly(y - z, x, y, z, domain = 'QQ')
f3 = Poly(-2*z**3 - z**2, x, y, z, domain = "QQ")

f = [f0, f1, f2, f3]
temp = calS(f1,f2,t="lex")
myDiv(f, temp, t="lex", gen=(x,y,z), dom="QQ")

([Poly(1, x, y, z, domain='QQ'), 0, Poly(1, x, y, z, domain='QQ'), 0],
 2*z**2 + z)

In [13]:
f0 = Poly(x*z - y, x, y, z, domain = 'QQ')
f1 = Poly(x*y + 2*z**2, x, y, z, domain = 'QQ')
f2 = Poly(y - z, x, y, z, domain = 'QQ')
f3 = Poly(-2*z**3 - z**2, x, y, z, domain = "QQ")
f4 = Poly(2*z**2 + z, x, y, z, domain = "QQ")

f = [f0, f1, f2, f3, f4]
temp = calS(f3,f4,t="lex")
myDiv(f, temp, t="lex", gen=(x,y,z), dom="QQ")

([0, 0, 0, 0, 0], 0)

In [14]:
#02

f0 = Poly(x*z - y, x, y, z, domain = 'QQ')
f1 = Poly(x*y + 2*z**2, x, y, z, domain = 'QQ')
f2 = Poly(y - z, x, y, z, domain = 'QQ')

f = [f0, f1, f2]
print(f)
print("---myGroebner---")
G = myGroebner(f, t = "lex", gen =(x,y,z), dom='QQ')

[Poly(x*z - y, x, y, z, domain='QQ'), Poly(x*y + 2*z**2, x, y, z, domain='QQ'), Poly(y - z, x, y, z, domain='QQ')]
---myGroebner---
g0 = x z - y
g1 = x y + 2 z^{2}
g2 = y - z
g3 = - 2 z^{3} - z^{2}
g4 = 2 z^{2} + z


In [47]:
#Example 01 (02.07)
f0 = Poly(x**3 - 2*x*y, x, y, z, domain = 'QQ')
f1 = Poly(x**2*y - 2*y**2 + x, x, y, z, domain = 'QQ')

f = [f0, f1]

f2 = Poly(-x**2, x, y, z, domain = 'QQ')
f3 = Poly(-2*x*y, x, y, z, domain = 'QQ')
f4 = Poly(-2*y**2 + x, x, y, z, domain = 'QQ')

myDiv(f, f2, t="lex", gen=(x,y,z), dom="QQ")
#myDiv(f, f3, t="lex", gen=(x,y,z), dom="QQ")
#myDiv(f, f4, t="lex", gen=(x,y,z), dom="QQ")
#sym.reduced(f2, f)
#sym.reduced(f3, f)
#sym.reduced(f4, f)

([0, 0], -x**2)